In [6]:
import pandas as pd
import numpy as np
import re

def to_snake(s: str) -> str:
    if not isinstance(s, str):
        return s
    s = s.strip().lower()
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\s+", "_", s)
    return s

def normalize_station_key(name: str) -> str:
    """Normalize station names for robust joining across files."""
    if pd.isna(name):
        return np.nan
    s = str(name).lower().strip()
    s = re.sub(r"[^\w\s]", " ", s)
    # Drop common noise words
    noise = ["charging", "ev", "station", "charger", "fast", "slow", "dc", "ac"]
    tokens = [t for t in s.split() if t not in noise]
    s = " ".join(tokens)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def series_mode(x: pd.Series):
    """Most frequent value; returns NaN if empty."""
    if x.empty:
        return np.nan
    counts = x.value_counts(dropna=True)
    return counts.index[0] if not counts.empty else np.nan

def haversine_km(lat1, lon1, lat2, lon2):
    """Great-circle distance in km (vectorized friendly)."""
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c


In [7]:
# File paths (keep names identical to your uploads)
p1 = "Files\\Cleaned dataset\\EV Car Power Consumption for Mumbai.csv"
p2 = "Files\\Cleaned dataset\\EV_Station_Location.csv"
p3 = "Files\\Cleaned dataset\\cleaned_combined_dataset.csv"

df1 = pd.read_csv(p1)
df2 = pd.read_csv(p2)
df3 = pd.read_csv(p3)

# Standardize column names
df1.columns = [to_snake(c) for c in df1.columns]
df2.columns = [to_snake(c) for c in df2.columns]
df3.columns = [to_snake(c) for c in df3.columns]

# Ensure session base has the fields we need (df3 is the base)
needed = ["energy_consumed_kwh", "charging_duration_hours", "charging_rate_kw",
          "vehicle_model", "time_of_day", "day_of_week", "charger_type", "user_type"]
for col in needed:
    if col not in df3.columns and col in df1.columns and len(df3) == len(df1):
        df3[col] = df1[col].values

# Ensure there is a station_name to link on
if "station_name" not in df3.columns and "station_name" in df1.columns:
    df3["station_name"] = df1["station_name"]
if "station_name" not in df3.columns and "charging_station_location" in df3.columns:
    df3["station_name"] = df3["charging_station_location"]

# Build join keys
df3["station_key"] = df3["station_name"].apply(normalize_station_key) if "station_name" in df3.columns else np.nan
df2["station_key"] = df2["station_name"].apply(normalize_station_key) if "station_name" in df2.columns else np.nan

# Merge sessions with station location/geo metadata
loc_cols = [c for c in ["station_key","station_name","address","operator","charging_types",
                        "payment_options","amenities","latitude","longitude","city","state"] if c in df2.columns]
sessions_master = df3.merge(
    df2[loc_cols].drop_duplicates("station_key"),
    on="station_key",
    how="left",
    suffixes=("", "_loc")
)

# Save
sessions_master.to_csv("sessions_master.csv", index=False)
print("sessions_master.csv ->", sessions_master.shape)
sessions_master.head(3)


sessions_master.csv -> (262, 28)


,vehicle_model,battery_capacity_kwh,charging_station_id,charging_station_location,energy_consumed_kwh,charging_duration_hours,charging_rate_kw,time_of_day,day_of_week,state_of_charge_start_,...,station_name_loc,address,operator_loc,charging_types,payment_options,amenities,latitude,longitude,city,state
0,TATA Tiago,108.463007,Station_391,"Mulund West, Mumbai",60.712346,1.67,36.389181,Evening,Tuesday,29.371576,...,ATUM Charge - Mulund,"Mulund West, Mumbai",ATUM,"AC 7.4kW, DC 25kW","App, Card",Parking,19.1721,72.9568,Mumbai,Maharashtra
1,Hyundai Kona,50.000000,Station_327,"Juhu Tara Rd, Near Juhu Beach, Mumbai, Maharas...",79.457824,2.42,32.882870,Evening,Saturday,83.120003,...,EV Point - Juhu,"Juhu Tara Rd, Near Juhu Beach, Mumbai, Maharas...",EV Point,"AC 11kW, DC 22kW","App, Card",Parking,19.0994,72.8269,Mumbai,Maharashtra
2,MG Comet,85.000000,Station_162,"Ambernath Station, Thane",36.862140,1.41,26.185188,Evening,Friday,60.751781,...,ChargeZone - Ambernath,"Ambernath Station, Thane",ChargeZone,"AC 22kW, DC 50kW","App, Card","Parking, UPI",19.1860,73.1891,Mumbai,Maharashtra


In [8]:
# Build aggregates
agg_dict = {}
if "energy_consumed_kwh" in sessions_master.columns:
    agg_dict["energy_consumed_kwh"] = ["sum", "mean"]
if "charging_duration_hours" in sessions_master.columns:
    agg_dict["charging_duration_hours"] = "mean"
if "charging_rate_kw" in sessions_master.columns:
    agg_dict["charging_rate_kw"] = "mean"

group_cols = ["station_key"] + [c for c in ["station_name", "operator"] if c in sessions_master.columns]
station_agg = (
    sessions_master
    .groupby(group_cols, dropna=False)
    .agg(agg_dict)
    .reset_index()
)
# Flatten multi-index columns
station_agg.columns = ["_".join(col).rstrip("_") if isinstance(col, tuple) else col for col in station_agg.columns]

# Session count
station_counts = sessions_master.groupby("station_key", dropna=False).size().reset_index(name="sessions_count")
station_agg = station_agg.merge(station_counts, on="station_key", how="left")

# Popular vehicle model & peak time/day (if present)
if "vehicle_model" in sessions_master.columns:
    pop_model = (sessions_master.groupby("station_key")["vehicle_model"]
                 .agg(series_mode).reset_index().rename(columns={"vehicle_model":"popular_vehicle_model"}))
    station_agg = station_agg.merge(pop_model, on="station_key", how="left")

if "time_of_day" in sessions_master.columns:
    peak_time = (sessions_master.groupby("station_key")["time_of_day"]
                 .agg(series_mode).reset_index().rename(columns={"time_of_day":"peak_time_of_day"}))
    station_agg = station_agg.merge(peak_time, on="station_key", how="left")

if "day_of_week" in sessions_master.columns:
    peak_day = (sessions_master.groupby("station_key")["day_of_week"]
                .agg(series_mode).reset_index().rename(columns={"day_of_week":"peak_day_of_week"}))
    station_agg = station_agg.merge(peak_day, on="station_key", how="left")

# Attach geo fields (one per station_key)
geo_cols = [c for c in ["latitude","longitude","city","state","address","charging_types","amenities"] if c in sessions_master.columns]
geo_frame = sessions_master[["station_key"] + geo_cols].drop_duplicates("station_key")
station_agg = station_agg.merge(geo_frame, on="station_key", how="left")

# Density feature: stations within 2 km (requires lat/lon)
if "latitude" in station_agg.columns and "longitude" in station_agg.columns:
    lat = station_agg["latitude"].values
    lon = station_agg["longitude"].values
    within2 = []
    for i in range(len(station_agg)):
        if pd.isna(lat[i]) or pd.isna(lon[i]):
            within2.append(np.nan)
            continue
        dists = haversine_km(lat[i], lon[i], lat, lon)
        within2.append(int(((dists > 0) & (dists <= 2)).sum()))
    station_agg["stations_within_2km"] = within2

# Unmatched report
if "latitude" in sessions_master.columns:
    unmatched = sessions_master[sessions_master["latitude"].isna()]
else:
    unmatched = sessions_master.iloc[0:0].copy()
unmatched = unmatched[[c for c in ["station_name","operator","station_key"] if c in unmatched.columns]].drop_duplicates()

# Save
station_agg.to_csv("station_aggregates.csv", index=False)
unmatched.to_csv("unmatched_stations.csv", index=False)

print("station_aggregates.csv ->", station_agg.shape)
print("unmatched_stations.csv ->", unmatched.shape)
station_agg.head(5)


station_aggregates.csv -> (154, 19)
unmatched_stations.csv -> (0, 3)


,station_key,station_name,operator,energy_consumed_kwh_sum,energy_consumed_kwh_mean,charging_duration_hours_mean,charging_rate_kw_mean,sessions_count,popular_vehicle_model,peak_time_of_day,peak_day_of_week,latitude,longitude,city,state,address,charging_types,amenities,stations_within_2km
0,ather grid bandra west,Ather Grid - Bandra West,Ather,41.157014,41.157014,0.890000,46.129689,1,TATA Tiago,Morning,Friday,19.0618,72.8330,Mumbai,Maharashtra,"Linking Road, Opp National College, Bandra Wes...",DC 25kW,Street Parking,2
1,ather grid hiranandani,Ather Grid - Hiranandani,Ather,213.120594,71.040198,4.036667,18.247259,3,TATA Tiago,Evening,Friday,19.1206,72.9188,Mumbai,Maharashtra,"Hiranandani Gardens, Powai, Mumbai",DC Fast 25kW,Parking,5
2,ather grid sewri,Ather Grid - Sewri,Ather,21.134045,21.134045,0.650000,32.292046,1,Hyundai Kona,Night,Sunday,18.9959,72.8532,Mumbai,Maharashtra,"Next to VRL Logistics, Ashok Gardens, PD’Mello...",DC Fast 25kW,"Parking, App Connect",3
3,atum charge borivali,ATUM Charge - Borivali,ATUM,110.371533,55.185766,2.510000,30.206715,2,TATA Curvv,Afternoon,Friday,19.2298,72.8606,Mumbai,Maharashtra,"Borivali West, Mumbai","AC 7.4kW, DC 25kW",Parking,3
4,atum charge ghatkopar,ATUM Charge - Ghatkopar,ATUM,48.502887,48.502887,1.110000,43.609707,1,Hyundai Kona,Night,Monday,19.0852,72.9114,Mumbai,Maharashtra,"Ghatkopar East, Mumbai","AC 7.4kW, DC 25kW",Parking,3
